In [1]:
import torchvision

In [2]:
vgg16_true=torchvision.models.vgg16(pretrained=True)

/root/miniconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/miniconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
from torch.nn import Linear, Conv2d, MaxPool2d,Flatten
from torch import nn
#vgg16_true.add_module('add_linear',nn.Linear(1000,10))
#vgg16_true.classifier[7]=nn.Linear(1000,10)



# 创建新的线性层
new_linear = nn.Linear(1000, 10)
# 将新的线性层添加到分类器中
vgg16_true.classifier.add_module('7', new_linear)
print(vgg16_true)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
import torch
import torchvision
from tensorboardX import SummaryWriter
from torch import nn
from torch.nn import Linear, Conv2d, MaxPool2d,Flatten
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
import numpy

#定义训练设备
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
#准备数据集
train_data=torchvision.datasets.CIFAR10("../dataset",train=True,transform=torchvision.transforms.ToTensor(),
                                     download=True)
test_data=torchvision.datasets.CIFAR10("../dataset",train=False,transform=torchvision.transforms.ToTensor(),
                                     download=True)
train_data_size=len(test_data)
test_data_size=len(test_data)
print("训练数据集长度：{}\n测试数据集长度：{}".format(train_data,test_data))

#加载数据集
train_dataloader=DataLoader(train_data,batch_size=64)
test_dataloader=DataLoader(test_data,batch_size=64)

#搭建神经网络

#创建网络模型
# class Model(nn.Module):
#     def __init__(self):
#         super(Model, self).__init__()
#         self.model1=nn.Sequential(
#             Conv2d(3,32,5,stride=1,padding=2),
#             MaxPool2d(2),
#             Conv2d(32, 32, 5, padding=2),
#             MaxPool2d(2),
#             Conv2d(32, 64, 5, padding=2),
#             MaxPool2d(2),
#             Flatten(),
#             Linear(1024,64),
#             Linear(64,10),
#         )

#     def forward(self,x):
#         x=self.model1(x)
#         return x


model=vgg16_true
model.to(device)
#损失函数
loss_fn=nn.CrossEntropyLoss()
loss_fn.to(device)
#定义优化器
learning_rate=0.001
#1e-2=1*10^(-2)=1/100=0.01
optimizer=torch.optim.SGD(vgg16_true.parameters(),lr=learning_rate,momentum=0.9)

#设置训练网络参数

#记录训练次数
total_train_step=0
#记录测试次数
total_test_step=0
#训练轮数
epoch=20

#添加Tensorboard
writer=SummaryWriter("../tf-logs")

for i in range(epoch):
    print("------第{}轮训练开始------".format(i+1))

    model.train()
    #训练步骤开始
    for data in train_dataloader:
        imgs,targets=data
        imgs=imgs.to(device)
        targets=targets.to(device)
        outputs=model(imgs)
        #计算loss
        loss=loss_fn(outputs,targets)
        #清零梯度
        optimizer.zero_grad()
        #反向传播
        loss.backward()
        #进行优化
        optimizer.step()
        if total_train_step%100==0:
            print("训练次数{},loss:{}".format(total_train_step,loss.item()))
            writer.add_scalar("train_loss",loss.item(),total_train_step)
        total_train_step = total_train_step + 1

    #测试步骤开始，取消梯度计算
    model.eval()
    total_test_loss=0
    total_accuracy=0
    #对现有模型进行测试
    with torch.no_grad():
        for data in test_dataloader:
            imgs,targets=data
            imgs = imgs.to(device)
            targets = targets.to(device)
            outputs=model(imgs)
            loss=loss_fn(outputs,targets)
            #loss是tensor数据类型，加上item()
            total_test_loss=total_test_loss+loss.item()
            #计算正确率
            accuracy=(outputs.argmax(1)==targets).sum()
            total_accuracy=total_accuracy+accuracy

        print("整体测试集上的loss：{}".format(total_test_loss))
        print("整体测试集上的正确率：{}".format(total_accuracy/test_data_size))
        writer.add_scalar("test_loss",total_test_loss,total_test_step)
        writer.add_scalar("test_accuracy",total_accuracy/test_data_size,total_test_step)
        total_test_step=total_test_step+1

        #保存训练模型
        if total_train_step%5==0:
            torch.save(model,"model_vgg16_{}.pth".format(total_test_step))
            #torch.save(model.state_dict(), "model_{}.pth".format(total_test_step))
            print("第{}轮训练的模型已保存".format(total_test_step))

#writer.close()

cuda
Files already downloaded and verified
Files already downloaded and verified
训练数据集长度：Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ../dataset
    Split: Train
    StandardTransform
Transform: ToTensor()
测试数据集长度：Dataset CIFAR10
    Number of datapoints: 10000
    Root location: ../dataset
    Split: Test
    StandardTransform
Transform: ToTensor()
------第1轮训练开始------
训练次数0,loss:3.6503617763519287
训练次数100,loss:0.9427316784858704
训练次数200,loss:0.720602810382843
训练次数300,loss:0.8829635977745056
训练次数400,loss:0.6166918873786926
训练次数500,loss:0.7819783091545105
训练次数600,loss:0.7439218759536743
训练次数700,loss:0.8468116521835327
整体测试集上的loss：88.25862419605255
整体测试集上的正确率：0.8098999857902527
------第2轮训练开始------
训练次数800,loss:0.5802260041236877
训练次数900,loss:0.48069965839385986
训练次数1000,loss:0.5475772619247437
训练次数1100,loss:0.4974369704723358
训练次数1200,loss:0.4773380756378174
训练次数1300,loss:0.3559010624885559
训练次数1400,loss:0.2656080722808838
训练次数1500,loss:0.48887816071510315
整体测试集上的lo